In [1]:
from pathlib import Path
import pybedtools

This notebook documents the process for extending Dorina regulators database.
Dorina depends on the correct naming of both the bedfile and and bed record (the rows in the bed file). Dorina filter most bed files by the name, and if the name of the file and the bed record do not match, there will be problems. Please see dorina.regulator.py line 80 for implementation details. 

Also, webdorina depends on a specific parsing to process the regulator site. This specification is:  
'{experiment}#{reg_name}_{assembly}*{reg_name}'  
PARCLIP#TAF15_hg19*TAF15

To keep the database consistent with the old data 
- peaks in experimental replicates are merged
- the score of the resulting peak is the median peak score

This may not be optimal for every experiment, but the database contains various types of experiments.

In [2]:
def change_name(rec, new_name=""):
    rec.name = new_name
    return rec

## Fix for mm10 regulators on 13 April 2018

dorina and webdorina were not working with mm10 regulatos due to mismatch of the regulators record name and .bed file name.  
This section fixes this issue

In [3]:
bed_files = Path('/Volumes/prj/dorina2/regulators/m_musculus/mm10/').glob('*.bed')

In [5]:
for bed_file in bed_files:
    bt = pybedtools.BedTool(str(bed_file))
    new_name = bed_file.stem + '*'
    bt.each(change_name, new_name=new_name).saveas().moveto(str(bed_file))

 there are still problems with the pictar files